In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [7]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(1, 100, 5)
non_lin = [0]

alpha = 0
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    bar.update()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    #stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
       
        if l == 0:
            nonlin = tf.exp
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = 1, reg = 'l2', 
                                lr = 1e-4, train_params = False, scale_init = 1, bias_init = 0)
        
        loss, cv = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], 
                             max_iters = 60) 
        
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                100%
[#                   ] | ETA: 00:00:00

new best model for cell  517442753 . r2 test:  -0.0086183517619
new best model for cell  517444738 . r2 test:  0.0107385504348
new best model for cell  517443331 . r2 test:  0.00617195171319
new best model for cell  517442436 . r2 test:  0.0227825209241
new best model for cell  517445575 . r2 test:  -0.0101050828017
new best model for cell  517444426 . r2 test:  0.0376963221961
new best model for cell  517442471 . r2 test:  0.0313046776186
new best model for cell  517443181 . r2 test:  0.0113933560948
new best model for cell  517443087 . r2 test:  0.0260468094513
new best model for cell  517446833 . r2 test:  0.0321968469877
new best model for cell  517443827 . r2 test:  0.0765867286044
new best model for cell  517443924 . r2 test:  0.042086881749
new best model for cell  517443957 . r2 test:  0.0126140425323
new best model for cell  517442722 . r2 test:  0.0503405884704
new best model for cell  517446488 . r2 test:  0.00745804946565
new best model for cell  517442675 . r2 test:  0.017

[##                  ] | ETA: 00:14:16

 517443571 . r2 test:  0.0265699493069
new best model for cell  517442753 . r2 test:  0.00458145095057
new best model for cell  517445575 . r2 test:  -0.00749606910261
new best model for cell  517443087 . r2 test:  0.0292670845974
new best model for cell  517446488 . r2 test:  0.0159121376058
new best model for cell  517446554 . r2 test:  -0.00532208190573
new best model for cell  517444175 . r2 test:  0.0278583539456
new best model for cell 

[###                 ] | ETA: 00:21:58

...

[#######             ] | ETA: 00:33:38

 517446554 . r2 test:  0.0247683599916
new best model for cell 

[####################] | ETA: 00:00:00

 517442753 . r2 test:  0.0431760374883



Total time elapsed: 02:21:54


In [8]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
        print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n
        
    print best_model[c]

explained var:  0.043 non_lin: 0 n_components 31
(0.043176037488321585, (0, 31))
explained var:  0.011 non_lin: 0 n_components 1
(0.010738550434807797, (0, 1))
explained var:  0.006 non_lin: 0 n_components 1
(0.0061719517131909457, (0, 1))
explained var:  0.023 non_lin: 0 n_components 1
(0.022782520924060714, (0, 1))
explained var:  -0.006 non_lin: 0 n_components 11
(-0.0056844161105775691, (0, 11))
explained var:  0.038 non_lin: 0 n_components 1
(0.037696322196121498, (0, 1))
explained var:  0.031 non_lin: 0 n_components 1
(0.031304677618642016, (0, 1))
explained var:  0.011 non_lin: 0 n_components 1
(0.011393356094846435, (0, 1))
explained var:  0.029 non_lin: 0 n_components 6
(0.029267084597371543, (0, 6))
explained var:  0.032 non_lin: 0 n_components 1
(0.03219684698767012, (0, 1))
explained var:  0.077 non_lin: 0 n_components 1
(0.07658672860441966, (0, 1))
explained var:  0.042 non_lin: 0 n_components 1
(0.042086881748984263, (0, 1))
explained var:  0.02 non_lin: 0 n_components 1

In [10]:
pkl_file = open('./boc/formatted/LN_Exp', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()
